In [1]:
using COSMO, LinearAlgebra, Random

In [2]:
# Non-negative. Generally an indefinite matrix, but solve for
# PSD X as max(Tr AX), this PSD program is a linear program with
# conic constraint.
A = [1 0.1 0.2 0.05; 0.2 1 0.5 0.01; 0.3 0.2 1 0.05; 0.01 0.5 0.05 1]
k = 2
n = size(A, 2)
N = n * n
A

4×4 Matrix{Float64}:
 1.0   0.1  0.2   0.05
 0.2   1.0  0.5   0.01
 0.3   0.2  1.0   0.05
 0.01  0.5  0.05  1.0

In [3]:
Matrix(1.0I, N, N)[begin:n:end, :]

4×16 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  1.0  0.0  0.0  0.0

In [4]:
[zeros(1, N) 1; Matrix(1.0I, N, N)[begin:n:end, :] zeros(n, 1)]

5×17 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0

In [5]:
model = COSMO.Model()
psd_c = COSMO.Constraint(Matrix(1.0I, N, N), zeros(N), COSMO.PsdCone)
box_c = COSMO.Constraint(
    Matrix(1.0I, N, N), zeros(N), COSMO.Box(zeros(N), ones(N)))
# Elements equal to 1 or 0. Sum of squares (||v||_2^2) equal to k.
l2_c = COSMO.Constraint(
    vec(Matrix(1.0I, n, n))', [-Float64(k)], COSMO.ZeroSet(1))
# One row: The diag entry may be zero or nonzero.
row_c = repeat(Matrix(-1.0I, n, n), inner=(1, n))
for i = 1:n
    row_c[i, 1 + (n+1)*(i-1)] = k-1
end
row_c = COSMO.Constraint(row_c, zeros(n), COSMO.ZeroSet(n))
assemble!(model, zeros(N, N), -vec(A), [psd_c, box_c, l2_c, row_c])

In [6]:
result = COSMO.optimize!(model)

>>> COSMO - Results
Status: 

Solved
Iterations: 25 (incl. 0 safeguarding iterations)
Optimal Objective: -2.7
Runtime: 3086.68ms
Setup Time: 817.39ms

Avg Iter Time: 56.61ms

In [7]:
reshape(result.x, n, n)

4×4 Matrix{Float64}:
 -7.71537e-9  -2.84948e-9  -2.05334e-9   2.04365e-9
 -3.01474e-9   1.0          1.0         -3.69062e-9
 -7.21494e-9   1.0          1.0          2.50193e-9
  2.51798e-9   1.4283e-8    1.81136e-9   8.50403e-10

In [8]:
# Non-negative. Generally an indefinite matrix, but solve for
# PSD X as max(Tr AX), this PSD program is a linear program with
# conic constraint.
n = 101
A = abs.(randn(n, n))
k = 3
N = n * n;

In [38]:
model = COSMO.Model()
psd_c = COSMO.Constraint(Matrix(1.0I, N, N), zeros(N), COSMO.PsdCone)
box_c = COSMO.Constraint(
    Matrix(1.0I, N, N), zeros(N), COSMO.Box(zeros(N), ones(N)))
# Elements equal to 1 or 0. Sum of squares (||v||_2^2) equal to k.
l2_c = COSMO.Constraint(
    vec(Matrix(1.0I, n, n))', [-Float64(k)], COSMO.ZeroSet(1))
# One row: The diag entry may be zero or nonzero.
row_c = repeat(Matrix(-1.0I, n, n), inner=(1, n))
for i = 1:n
    row_c[i, 1 + (n+1)*(i-1)] = k-1
end
row_c = COSMO.Constraint(row_c, zeros(n), COSMO.ZeroSet(n))
assemble!(model, zeros(N, N), -vec(A), [psd_c, box_c, l2_c, row_c],
    settings = COSMO.Settings(eps_abs=0.0001, eps_rel=0))

In [39]:
result = COSMO.optimize!(model)

>>> COSMO - Results
Status: 

Solved
Iterations: 3600 (incl. 0 safeguarding iterations)
Optimal Objective: -16.86
Runtime: 17959.99ms
Setup Time: 22.76ms

Avg Iter Time: 4.98ms

In [40]:
maximum(model.δx)

3.373277475948733e-5

In [41]:
sort(abs.(result.x))[end-100:end]

101-element Vector{Float64}:
 0.0007637370747083512
 0.0007677750300654116
 0.0007792639417449263
 0.0008003540823541552
 0.0008087531192765188
 0.0008396185079572151
 0.0008459974779157325
 0.0008478888128824716
 0.0008525810032799002
 0.0008650072370124195
 0.0009699952902611318
 0.0009869231437383366
 0.0010038711665317059
 ⋮
 0.06529316187854386
 0.06529644325269746
 0.11647811245240916
 0.8252827743397054
 0.8252854734338155
 0.8884678811440803
 0.8981210037418697
 0.8981337746457158
 0.9039173427288025
 0.9039328127267489
 0.9142094712019462
 0.9422970949383201